# 3d mitochondria visualizer using vtk 
- Includes a Neuroglancer link generator and a vtk viewer centered on a mitoid of interest 
- This notebook is modified from Allen Institute tutorial on working with Meshes by Forrest Collman  
The original tutorial is located here:   https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/intro/MeshExample.ipynb

## Python Requirements
You will need to run this notebook in a Python 3.7 environment  
Sometime meshparty will have a conflict with other tools and not work properly in Anaconda  
If that happens, you will need to start over with a new Anaconda environment  
Install allensdk first, then meshparty, then any other other tools desired  
The installation described below worked well:  
* Install new environment v3.7 in Anaconda
* conda install jupyter
* pip install allensdk
* pip install meshparty
* pip install caveclient
* pip install 'itkwidgets[notebook]>=1.0a8'
* pip install --upgrade --pre itk
* pip install gdown

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

In [3]:
import vtk

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
# this csv file is 268 MB and too large to place on Github
# in order to run this notebook, you must first download the file and place on your local machine in the /data folder
# the csv file is available here: https://zenodo.org/record/5579388/files/211019_mitochondria_info.csv

mito = pd.read_csv('data/211019_mitochondria_info.csv')
mito

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
0,3384540,5916,103764,47040,103,103734,47016,102,103798,47066,107,648518346348124201,371475.12,168403.20,4120
1,2526419,1075376,87582,60964,1435,87204,59752,1215,87992,62504,1609,648518346346303282,313543.56,218251.12,57400
2,1379480,483500,65740,73550,392,65556,73322,292,65976,73880,509,648518346341355778,235349.20,263309.00,15680
3,3380073,23140,103750,46904,176,103704,46862,173,103812,46946,181,648518346348124771,371425.00,167916.32,7040
4,1783610,11996,75124,43042,2100,75088,43008,2097,75164,43086,2103,648518346342925287,268943.92,154090.36,84000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409611,2864896,3878784,93134,59860,836,93000,59406,798,93294,60294,867,648518346349536888,333419.72,214298.80,33440
2409612,2161774,4075748,80312,56522,1814,79536,55348,1714,80940,57054,1889,648518346349524070,287516.96,202348.76,72560
2409613,2753701,4576444,90324,60436,2037,89490,60120,1851,90750,61138,2142,648518346341354380,323359.92,216360.88,81480
2409614,1963708,5805612,75674,72546,744,75266,71628,670,76032,73302,795,648518346343047176,270912.92,259714.68,29760


In [6]:
astrocytes = pd.read_csv('data/astrocytes.csv', index_col=[0])
astrocytes.head()

,centroid_x,centroid_y,centroid_z,cell_segid,cell_type,cell_subtype
0,88240,60656,256,648518346349517321,glia,astrocyte
1,72368,56912,590,648518346341392909,glia,astrocyte
4,72704,54688,506,648518346349536799,glia,astrocyte
5,74576,75520,1163,648518346349525537,glia,astrocyte
6,79328,67360,445,648518346349525544,glia,astrocyte


In [7]:
astrocyte_list = astrocytes.cell_segid.to_list()
print(astrocyte_list)

[648518346349517321, 648518346341392909, 648518346349536799, 648518346349525537, 648518346349525544, 648518346349526583, 648518346349490239, 648518346349524070, 648518346349386860, 648518346349508722, 648518346349536888, 648518346349516953, 648518346349521062, 648518346342468778, 648518346342795947, 648518346349524139, 648518346342917290, 648518346349528249, 648518346349528250, 648518346349507266, 648518346349498574, 648518346349528271, 648518346349525715, 648518346341399264, 648518346349536487, 648518346349532924, 648518346349527316, 648518346349527317, 648518346349527319, 648518346342807831, 648518346349537050, 648518346349538089, 648518346341356348, 648518346349517141, 648518346349539158, 648518346349525862, 648518346344075119, 648518346341354380, 648518346349528465, 648518346349484948, 648518346342797245, 648518346349521344, 648518346342795202, 648518346349530569]


# Use index value to view an astrocyte from list 

In [8]:
# pull all mitos from a cellid of interest from astrocyte list, by index number

indx = 28
cellid = astrocyte_list[indx]

mito_querydf = mito[mito['cellid'] == cellid]
mito_querydf.sort_values(by='mito_vx', ascending=False)


,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
8141,866030,100327168,64014,42198,1606,56972,36422,1335,69342,47894,2063,648518346349527319,229170.12,151068.84,64240
1194,1083380,42615580,67652,42956,1396,62308,39686,1076,73426,46952,1714,648518346349527319,242194.16,153782.48,55840
10972,854433,26071428,60272,40094,1048,56888,39172,560,63680,41540,1312,648518346349527319,215773.76,143536.52,41920
2211,1083414,16229984,64066,38346,1226,62378,36836,1121,68092,40178,1312,648518346349527319,229356.28,137278.68,49040
50903,1083667,15729544,65638,39818,1190,62714,37810,1041,68520,41062,1311,648518346349527319,234984.04,142548.44,47600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90280,1423219,1360,67990,40050,1201,67978,40036,1199,68006,40066,1203,648518346349527319,243404.20,143379.00,48040
84274,1087694,1272,62292,41402,1471,62274,41390,1469,62306,41418,1473,648518346349527319,223005.36,148219.16,58840
82950,1203810,1252,64340,43282,1455,64326,43272,1452,64354,43296,1458,648518346349527319,230337.20,154949.56,58200
82703,1082518,1248,61190,39616,1397,61164,39606,1394,61222,39628,1401,648518346349527319,219060.20,141825.28,55880


In [9]:
cellid

648518346349527319

In [10]:
thresh = 2000000

In [11]:
mito_query_abovethresh = mito_querydf[mito_querydf.mito_vx > thresh]
mito_query_abovethresh

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
1194,1083380,42615580,67652,42956,1396,62308,39686,1076,73426,46952,1714,648518346349527319,242194.16,153782.48,55840
2211,1083414,16229984,64066,38346,1226,62378,36836,1121,68092,40178,1312,648518346349527319,229356.28,137278.68,49040
4491,966479,3344056,60698,37578,1624,59988,36742,1490,61102,38362,1714,648518346349527319,217298.84,134529.24,64960
8141,866030,100327168,64014,42198,1606,56972,36422,1335,69342,47894,2063,648518346349527319,229170.12,151068.84,64240
8193,1304222,7678784,67924,38992,856,66382,38258,649,69666,39514,1066,648518346349527319,243167.92,139591.36,34240
10972,854433,26071428,60272,40094,1048,56888,39172,560,63680,41540,1312,648518346349527319,215773.76,143536.52,41920
13380,739489,13177320,59562,35984,1437,56486,35008,1326,61970,36888,1701,648518346349527319,213231.96,128822.72,57480
27111,855172,2124972,58648,39582,861,58404,38916,787,58934,39834,943,648518346349527319,209959.84,141703.56,34440
50903,1083667,15729544,65638,39818,1190,62714,37810,1041,68520,41062,1311,648518346349527319,234984.04,142548.44,47600
51628,1300104,2175716,65306,38682,1498,64892,37918,1468,65836,39558,1536,648518346349527319,233795.48,138481.56,59920


In [12]:
mito_query_belowthresh = mito_querydf[mito_querydf.mito_vx <= thresh]
mito_query_belowthresh

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
674,964170,89772,60356,37864,1091,60202,37760,1084,60458,37982,1099,648518346349527319,216074.48,135553.12,43640
2285,1209147,896072,64898,45928,1664,64784,45606,1627,65008,46130,1706,648518346349527319,232334.84,164422.24,66560
3627,1083417,7920,62388,40838,1234,62300,40782,1233,62442,40904,1238,648518346349527319,223349.04,146200.04,49360
3914,1074216,560604,62746,36692,1826,62498,36420,1807,62908,36918,1851,648518346349527319,224630.68,131357.36,73040
4181,1304771,4828,66886,39052,1020,66864,39016,1018,66914,39080,1023,648518346349527319,239451.88,139806.16,40800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409382,857091,1564688,58162,39586,1601,57814,39016,1554,58620,39892,1673,648518346349527319,208219.96,141717.88,64040
2409463,1533361,1732176,70414,37604,1289,69980,37312,1271,70934,38174,1316,648518346349527319,252082.12,134622.32,51560
2409502,1420116,1832208,67812,39974,846,67514,39224,759,68076,40396,970,648518346349527319,242766.96,143106.92,33840
2409534,1662641,1972060,72240,44154,1321,71518,43578,1284,72904,44434,1445,648518346349527319,258619.20,158071.32,52840


In [13]:
mito_query_abovethresh_mitolist = mito_query_abovethresh.mito_id.to_list()
print(f"length: "+str(len(mito_query_abovethresh_mitolist)))
print(f"type: "+str(type(mito_query_abovethresh_mitolist)))
print('')
# print(mito_query_abovethresh_mitolist) # uncomment to print the mito id list

length: 46
type: <class 'list'>



In [14]:
mito_query_belowthresh_mitolist = mito_query_belowthresh.mito_id.to_list()
print(f"length: "+str(len(mito_query_belowthresh_mitolist)))
print(f"type: "+str(type(mito_query_belowthresh_mitolist)))
print('')
# print(mito_query_belowthresh_mitolist) # uncomment to print the mito id list

length: 822
type: <class 'list'>



# vtk 3d viewer

In [15]:
# setup the mesh meta to handle downloads and caching
mesh_dir = 'data/neuron_meshes_v185/' # or change to your desired folder
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

# setup the mesh meta to handle downloads and caching
mito_mesh_dir = 'data/meshes/'
mito_source = "precomputed://https://tigerdata.princeton.edu/sseung-archive/pinky100-mito/seg_191220"
mito_mm = trimesh_io.MeshMeta(cv_path=mito_source,
                         disk_cache_path=mito_mesh_dir)

In [16]:
cell_id = cellid

In [17]:
# If you get an error, run the next cell to download the mesh, then rerun this cell again
seg_id = cell_id
mesh_file = os.path.join(mesh_dir, str(seg_id)+'.h5')
mesh = mm.mesh(filename = mesh_file)

In [18]:
# If you get an error on the previous cell, run this cell
# After the mesh is successfully downloaded, rerun the previous cell 
downloadmesh = mm.mesh(seg_id = seg_id, remove_duplicate_vertices=True)

In [19]:
len(mito_query_abovethresh_mitolist)

46

In [20]:
len(mito_query_belowthresh_mitolist)

822

### The following cell only needs to be run once
- You do not need to run this cell if you have previously downloaded the mito mesh files
- Note: this will take a long time, which gets longer the more mitos are in the cell of interest

In [21]:
# download the mito meshes for this cell

for i in range(len(mito_query_abovethresh_mitolist)):
    mito_id = mito_query_abovethresh_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)
    
for i in range(len(mito_query_belowthresh_mitolist)):
    mito_id = mito_query_belowthresh_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)


In [22]:
# iterate the mito_mesh for each mito
# from https://python-forum.io/thread-23500.html

var_iterator_abovethresh = {}
for i in range(len(mito_query_abovethresh_mitolist)):
    mito_seg_id = mito_query_abovethresh_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_abovethresh['mito_mesh_' + str(i)] = mito_mm.mesh(filename = mito_mesh_file)
    
locals().update(var_iterator_abovethresh)


var_iterator_belowthresh = {}
for i in range(len(mito_query_belowthresh_mitolist)):
    mito_seg_id = mito_query_belowthresh_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_belowthresh['mito_mesh_' + str(i)] = mito_mm.mesh(filename = mito_mesh_file)
    
locals().update(var_iterator_belowthresh)


In [23]:
len(var_iterator_abovethresh)

46

In [24]:
var_iterator_abovethresh

{'mito_mesh_0': <trimesh.Mesh(vertices.shape=(113878, 3), faces.shape=(227672, 3))>,
 'mito_mesh_1': <trimesh.Mesh(vertices.shape=(37699, 3), faces.shape=(75498, 3))>,
 'mito_mesh_2': <trimesh.Mesh(vertices.shape=(6960, 3), faces.shape=(13872, 3))>,
 'mito_mesh_3': <trimesh.Mesh(vertices.shape=(238024, 3), faces.shape=(476176, 3))>,
 'mito_mesh_4': <trimesh.Mesh(vertices.shape=(17356, 3), faces.shape=(34700, 3))>,
 'mito_mesh_5': <trimesh.Mesh(vertices.shape=(63311, 3), faces.shape=(126582, 3))>,
 'mito_mesh_6': <trimesh.Mesh(vertices.shape=(35277, 3), faces.shape=(70590, 3))>,
 'mito_mesh_7': <trimesh.Mesh(vertices.shape=(4946, 3), faces.shape=(9896, 3))>,
 'mito_mesh_8': <trimesh.Mesh(vertices.shape=(35925, 3), faces.shape=(71826, 3))>,
 'mito_mesh_9': <trimesh.Mesh(vertices.shape=(4790, 3), faces.shape=(9600, 3))>,
 'mito_mesh_10': <trimesh.Mesh(vertices.shape=(6489, 3), faces.shape=(12926, 3))>,
 'mito_mesh_11': <trimesh.Mesh(vertices.shape=(11881, 3), faces.shape=(23742, 3))>,
 'm

In [25]:
# opacity and color settings for cell membrane and mito meshes
cell_opac = 0.05 # 0.075
cell_color = (1, 1, 1) # (0.2, 0.4, 0.7) blue
mito_opac_abovethresh = 1 # 0.35
mito_color_abovethresh = (0.2, 0.4, 0.7)
mito_opac_belowthresh = 1
mito_color_belowthresh = (0.7, 0.2, 0.2)

# cell membrane mesh
mesh_actor = trimesh_vtk.mesh_actor(mesh, opacity=cell_opac, color=cell_color)

# mito meshes

actor_iterator_abovethresh = {}
for i in range(len(mito_query_abovethresh_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_abovethresh['mito_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(var_iterator_abovethresh['mito_mesh_'+str(i)], opacity=mito_opac_abovethresh, color=(mito_color_abovethresh))
    
locals().update(actor_iterator_abovethresh)

actor_iterator_belowthresh = {}
for i in range(len(mito_query_belowthresh_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_belowthresh['mito_mesh_actor_' + str(i+(len(var_iterator_abovethresh)))] = trimesh_vtk.mesh_actor(var_iterator_belowthresh['mito_mesh_'+str(i)], opacity=mito_opac_belowthresh, color=(mito_color_belowthresh))
    
locals().update(actor_iterator_belowthresh)

# update dictionary for the render actors code below
actor_iterator = {}
actor_iterator['mesh_actor'] = mesh_actor
actor_iterator.update(actor_iterator_abovethresh)
actor_iterator.update(actor_iterator_belowthresh)
locals().update(actor_iterator)

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=200) # 150

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000018D6DBDC7B0) at 0x0000018D378E5888>

In [26]:
#actor_iterator

In [27]:
#type(actor_iterator.values())

In [28]:
# save image to disk
# uncomment to save

#camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=190)

#trimesh_vtk.render_actors(actor_iterator.values(),
#                          filename='vtk_images/'+str(cellid)+'.png',
#                          do_save=True,
#                          scale=6,
#                          camera=camera
#                         )

In [29]:
cellid

648518346349527319

In [30]:
# 648518346349536888 has two to three astrocytes

In [31]:
# 648518346341354380 very interesting case

In [32]:
astrocyte_list

[648518346349517321,
 648518346341392909,
 648518346349536799,
 648518346349525537,
 648518346349525544,
 648518346349526583,
 648518346349490239,
 648518346349524070,
 648518346349386860,
 648518346349508722,
 648518346349536888,
 648518346349516953,
 648518346349521062,
 648518346342468778,
 648518346342795947,
 648518346349524139,
 648518346342917290,
 648518346349528249,
 648518346349528250,
 648518346349507266,
 648518346349498574,
 648518346349528271,
 648518346349525715,
 648518346341399264,
 648518346349536487,
 648518346349532924,
 648518346349527316,
 648518346349527317,
 648518346349527319,
 648518346342807831,
 648518346349537050,
 648518346349538089,
 648518346341356348,
 648518346349517141,
 648518346349539158,
 648518346349525862,
 648518346344075119,
 648518346341354380,
 648518346349528465,
 648518346349484948,
 648518346342797245,
 648518346349521344,
 648518346342795202,
 648518346349530569]

# Neuroglancer visualizer

In [33]:
lookup = cellid

In [34]:
query = mito[mito["cellid"] == lookup]

In [35]:
len(query)

868

In [36]:
query_list = query["mito_id"].to_list()
spacer = "%22%2C%22"
query_list_join = spacer.join(map(str,query_list))
baseURL = 'https://neuromancer-seung-import.appspot.com/'
nglink_a = '#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22'
nglink_b = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://https://tigerdata.princeton.edu/sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://https://tigerdata.princeton.edu/sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%22'
nglink_c = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://https://tigerdata.princeton.edu/sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B83222.921875%2C52981.34765625%2C824.9962768554688%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22perspectiveOrientation%22:%5B-0.00825042650103569%2C0.06130112707614899%2C-0.0012821174459531903%2C0.9980843663215637%5D%2C%22perspectiveZoom%22:3618.7659948513424%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22mitochondria%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D'
neuroglancerURL = baseURL + nglink_a + str(lookup) + nglink_b + query_list_join + nglink_c
print(neuroglancerURL)

https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22648518346349527319%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://https://tigerdata.princeton.edu/sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://https://tigerdata.princeton.edu/sseung-archive/pinky1

In [37]:
query

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
674,964170,89772,60356,37864,1091,60202,37760,1084,60458,37982,1099,648518346349527319,216074.48,135553.12,43640
1194,1083380,42615580,67652,42956,1396,62308,39686,1076,73426,46952,1714,648518346349527319,242194.16,153782.48,55840
2211,1083414,16229984,64066,38346,1226,62378,36836,1121,68092,40178,1312,648518346349527319,229356.28,137278.68,49040
2285,1209147,896072,64898,45928,1664,64784,45606,1627,65008,46130,1706,648518346349527319,232334.84,164422.24,66560
3627,1083417,7920,62388,40838,1234,62300,40782,1233,62442,40904,1238,648518346349527319,223349.04,146200.04,49360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409382,857091,1564688,58162,39586,1601,57814,39016,1554,58620,39892,1673,648518346349527319,208219.96,141717.88,64040
2409463,1533361,1732176,70414,37604,1289,69980,37312,1271,70934,38174,1316,648518346349527319,252082.12,134622.32,51560
2409502,1420116,1832208,67812,39974,846,67514,39224,759,68076,40396,970,648518346349527319,242766.96,143106.92,33840
2409534,1662641,1972060,72240,44154,1321,71518,43578,1284,72904,44434,1445,648518346349527319,258619.20,158071.32,52840


# Center vtk view at centroid of mitoid of interest

In [38]:
mitoid_query = 980814

In [39]:
pull_centroid = mito_querydf.reset_index()
pull_centroid = pull_centroid.index[pull_centroid['mito_id'] == mitoid_query].to_list()
pull_centroid[0]

688

In [40]:
mito_mesh_centroid_dict = {}
mito_seg_id = mitoid_query
mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
mito_mesh_centroid_dict['mito_mesh_centroid_' + str(pull_centroid[0])] = mito_mm.mesh(filename = mito_mesh_file)
locals().update(mito_mesh_centroid_dict)

In [41]:
mito_mesh_centroid_dict

{'mito_mesh_centroid_688': <trimesh.Mesh(vertices.shape=(7766, 3), faces.shape=(15540, 3))>}

In [42]:
mito_mesh_centroid_dict.get('mito_mesh_centroid_'+str(pull_centroid[0])).centroid

array([240587.19641369, 182791.42416041,  56425.44772235])

In [43]:
# opacity and color settings for cell membrane and mito meshes
cell_opac = 0.05 # 0.075
cell_color = (1, 1, 1) # (0.2, 0.4, 0.7) blue
mito_opac_abovethresh = 1 # 0.35
mito_color_abovethresh = (0.2, 0.4, 0.7)
mito_opac_belowthresh = 1
mito_color_belowthresh = (0.7, 0.2, 0.2)
backoff_value = 20

# cell membrane mesh
mesh_actor = trimesh_vtk.mesh_actor(mesh, opacity=cell_opac, color=cell_color)

# mito meshes

actor_iterator_abovethresh = {}
for i in range(len(mito_query_abovethresh_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_abovethresh['mito_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(var_iterator_abovethresh['mito_mesh_'+str(i)], opacity=mito_opac_abovethresh, color=(mito_color_abovethresh))
    
locals().update(actor_iterator_abovethresh)

actor_iterator_belowthresh = {}
for i in range(len(mito_query_belowthresh_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_belowthresh['mito_mesh_actor_' + str(i+(len(var_iterator_abovethresh)))] = trimesh_vtk.mesh_actor(var_iterator_belowthresh['mito_mesh_'+str(i)], opacity=mito_opac_belowthresh, color=(mito_color_belowthresh))
    
locals().update(actor_iterator_belowthresh)

# update dictionary for the render actors code below
actor_iterator = {}
actor_iterator['mesh_actor'] = mesh_actor
actor_iterator.update(actor_iterator_abovethresh)
actor_iterator.update(actor_iterator_belowthresh)
locals().update(actor_iterator)

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mito_mesh_centroid_dict.get('mito_mesh_centroid_'+str(pull_centroid[0])).centroid, backoff=backoff_value) # 150

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000018D6DBE5290) at 0x0000018D69A6CA68>

In [44]:
# save image to disk
# uncomment to save

#camera = trimesh_vtk.oriented_camera(mito_mesh_centroid_dict.get('mito_mesh_centroid_'+str(pull_centroid[0])).centroid, backoff=backoff_value)

#trimesh_vtk.render_actors(actor_iterator.values(),
#                          filename='vtk_images/'+str(cellid)+'_'+str(mitoid_query)+'.png',
#                          do_save=True,
#                          scale=6,
#                          camera=camera
#                         )